Source: https://stackoverflow.com/questions/13794754/logistic-regression-using-scipy

In [39]:
import numpy as np
import scipy as sp
import scipy.optimize

# import matplotlib as mpl
# import os

In [2]:
# prepare the data
data = np.loadtxt('./data/some_internet_example.csv', delimiter=',', skiprows=1)   # numpy import
vY = data[:, 0]                                                                    # target
mX = data[:, 1:]                                                                   # features

In [7]:
intercept = np.ones(mX.shape[0]).reshape(mX.shape[0], 1)      # column of 1's
mX = np.concatenate((intercept, mX), axis = 1)                # concatenate column of 1's with features
iK = mX.shape[1]                                              # number of columns
iN = mX.shape[0]                                              # number of rows

In [19]:
# logistic transformation
def logit(mX, vBeta):
    '''
    np.exp: exponential
    np.dot: array multiplication
    
    mX: array of features (plus constant column)
    vBeta: initial guess for coefficients
    '''
    return((np.exp(np.dot(mX, vBeta))/(1.0 + np.exp(np.dot(mX, vBeta)))))

In [26]:
# test function call
vBeta0 = np.array([-.10296645, -.0332327, -.01209484, .44626211,
                   .92554137, .53973828, 1.7993371, .7148045])
logit(mX, vBeta0)

array([0.20943741, 0.14987785, 0.3394435 , 0.49487821, 0.52281042,
       0.27651255, 0.14003313, 0.3601059 , 0.23458923, 0.27649193,
       0.36711555, 0.22973159, 0.76042806, 0.54964434, 0.3684585 ,
       0.3684585 , 0.29023828, 0.27107574, 0.45874142, 0.24730097,
       0.21566632, 0.23476508, 0.03570043, 0.24120645, 0.41785771,
       0.06868554, 0.31611378, 0.08056621, 0.23489326, 0.24193225,
       0.24193225, 0.48062344, 0.40811032, 0.08609919, 0.17463686,
       0.19969705, 0.26276004, 0.24449232, 0.08613498, 0.24118758,
       0.22787786, 0.07082728, 0.17550211, 0.09197414, 0.57368876,
       0.57368876, 0.08975177, 0.27049348, 0.13643796, 0.59921142,
       0.49003446, 0.25413638, 0.26219103, 0.29376793, 0.31291896,
       0.34834187, 0.70011523, 0.16635666, 0.337467  , 0.31682056,
       0.31682056, 0.2753156 , 0.29065659, 0.09275108, 0.68312574,
       0.44563593, 0.27834788, 0.1427219 , 0.44214783, 0.21201733,
       0.56526847, 0.33557457, 0.50152563, 0.2503423 , 0.28993

In [31]:
help(np.log)

Help on ufunc object:

log = class ufunc(builtins.object)
 |  Functions that operate element by element on whole arrays.
 |  
 |  To see the documentation for a specific ufunc, use `info`.  For
 |  example, ``np.info(np.sin)``.  Because ufuncs are written in C
 |  (for speed) and linked into Python with NumPy's ufunc facility,
 |  Python's help() function finds this page whenever help() is called
 |  on a ufunc.
 |  
 |  A detailed explanation of ufuncs can be found in the docs for :ref:`ufuncs`.
 |  
 |  Calling ufuncs:
 |  ===============
 |  
 |  op(*x[, out], where=True, **kwargs)
 |  Apply `op` to the arguments `*x` elementwise, broadcasting the arguments.
 |  
 |  The broadcasting rules are:
 |  
 |  * Dimensions of length 1 may be prepended to either array.
 |  * Arrays may be repeated along dimensions of length 1.
 |  
 |  Parameters
 |  ----------
 |  *x : array_like
 |      Input arrays.
 |  out : ndarray, None, or tuple of ndarray and None, optional
 |      Alternate array o

In [33]:
# cost function
def logLikelihoodLogit(vBeta, mX, vY):
    '''
    np.sum: sum all the values from the array (into a single value)
    np.log: log element-wise
    '''
    return(-(np.sum(vY*np.log(logit(mX, vBeta)) + (1-vY)*(np.log(1-logit(mX, vBeta))))))

logLikelihoodLogit(vBeta0, mX, vY) # test function call

102.17373263748843

In [40]:
# different parametrization of the cost function
def logLikelihoodLogitVerbose(vBeta, mX, vY):
    return(-(np.sum(vY*(np.dot(mX, vBeta) - np.log((1.0 + np.exp(np.dot(mX, vBeta))))) +
                    (1-vY)*(-np.log((1.0 + np.exp(np.dot(mX, vBeta))))))))
logLikelihoodLogitVerbose(vBeta0, mX, vY)  # test function call

# gradient function
def likelihoodScore(vBeta, mX, vY):
    return(np.dot(mX.T, 
                  (logit(mX, vBeta) - vY)))
likelihoodScore(vBeta0, mX, vY).shape # test function call
sp.optimize.check_grad(logLikelihoodLogitVerbose, likelihoodScore, 
                       vBeta0, mX, vY)  # check that the analytical gradient is close to 
                                                # numerical gradient

0.004206858522565068

In [41]:
# optimize the function (without gradient)
optimLogit = scipy.optimize.fmin_bfgs(logLikelihoodLogitVerbose, 
                                  x0 = np.array([-.1, -.03, -.01, .44, .92, .53,
                                            1.8, .71]), 
                                  args = (mX, vY), gtol = 1e-3)

# optimize the function (with gradient)
optimLogit = scipy.optimize.fmin_bfgs(logLikelihoodLogitVerbose, 
                                  x0 = np.array([-.1, -.03, -.01, .44, .92, .53,
                                            1.8, .71]), fprime = likelihoodScore, 
                                  args = (mX, vY), gtol = 1e-3)

Optimization terminated successfully.
         Current function value: 102.173733
         Iterations: 15
         Function evaluations: 210
         Gradient evaluations: 21
Optimization terminated successfully.
         Current function value: 102.173733
         Iterations: 15
         Function evaluations: 21
         Gradient evaluations: 21
